<a href="https://colab.research.google.com/github/rubenjimenech/chatbot/blob/main/ChatbotRubenJimenezipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder


In [5]:
import json
import pickle
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense
from sklearn.preprocessing import LabelEncoder

# load data from JSON file
with open('intents.json') as file:  # Reemplaza '/content/intents.json' con la ruta correcta a tu archivo JSON
    data = json.load(file)

# extract text and intent from data
texts = []
intents = []
for intent in data['intents']:
    for text in intent['patterns']:
        texts.append(text)
        intents.append(intent['tag'])

# tokenize text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
encoded_texts = tokenizer.texts_to_sequences(texts)

# save tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# pad sequences to have equal length
max_len = max([len(x) for x in encoded_texts])
padded_texts = pad_sequences(encoded_texts, maxlen=max_len, padding='post')

# create label encoder object
le = LabelEncoder()

# fit and transform the intents to integer labels
encoded_intents = le.fit_transform(intents)

# get the number of unique labels
num_intents = len(le.classes_)

# apply one-hot encoding to the integer labels
encoded_intents = tf.one_hot(encoded_intents, depth=num_intents)

# define model architecture
input_layer = Input(shape=(max_len,))
embedding_layer = Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128, input_length=max_len)(input_layer)
lstm_layer = LSTM(128)(embedding_layer)
output_layer = Dense(num_intents, activation='softmax')(lstm_layer)
model = Model(inputs=input_layer, outputs=output_layer)

# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# train model
model.fit(padded_texts, encoded_intents, epochs=25, batch_size=16)

# save model
model.save('chatbot_model.h5')


Epoch 1/25
12/12 [==============================] - 3s 24ms/step - loss: 3.9524 - accuracy: 0.0691
Epoch 2/25
12/12 [==============================] - 0s 40ms/step - loss: 3.9265 - accuracy: 0.0691
Epoch 3/25
12/12 [==============================] - 0s 26ms/step - loss: 3.8671 - accuracy: 0.0691
Epoch 4/25
12/12 [==============================] - 0s 29ms/step - loss: 3.7890 - accuracy: 0.0691
Epoch 5/25
12/12 [==============================] - 0s 41ms/step - loss: 3.7072 - accuracy: 0.0798
Epoch 6/25
12/12 [==============================] - 0s 30ms/step - loss: 3.5901 - accuracy: 0.0904
Epoch 7/25
12/12 [==============================] - 0s 24ms/step - loss: 3.5075 - accuracy: 0.0957
Epoch 8/25
12/12 [==============================] - 0s 38ms/step - loss: 3.4166 - accuracy: 0.0851
Epoch 9/25
12/12 [==============================] - 0s 33ms/step - loss: 3.3080 - accuracy: 0.0851
Epoch 10/25
12/12 [==============================] - 0s 28ms/step - loss: 3.1764 - accuracy: 0.0957
Epoch 11/

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [6]:
import json
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

# Load data from JSON file
with open('/content/intents.json') as file:
    data = json.load(file)

# Extract text patterns and intents from data
patterns = []
intents = []
for intent in data['intents']:
    for pattern in intent['patterns']:
        patterns.append(pattern)
        intents.append(intent['tag'])

# Tokenize text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(patterns)

# Load saved model
model = load_model('/content/chatbot_model.h5')

# Define maximum sequence length
max_len = model.input_shape[1]

# Create label encoder object
le = LabelEncoder()
le.fit(intents)

# Create inverse mapping of label encoder for intent prediction
intent_mapping = {i: label for i, label in enumerate(le.classes_)}

# Example usage:
# Suppose you have a sequence of text you want to classify:
input_text = "Recommend a history book"

# Tokenize and pad the input text
encoded_text = tokenizer.texts_to_sequences([input_text])
padded_text = pad_sequences(encoded_text, maxlen=max_len, padding='post')

# Predict intent
predicted = model.predict(padded_text)
predicted_label = intent_mapping[np.argmax(predicted)]

print(f"Predicted Intent: {predicted_label}")

1/1 [==============================] - 1s 559ms/step
Predicted Intent: History


In [7]:
mensajes_despedida=['adios','bye','hasta luego','good bye','stop','quit']
etiquetas_basicas=["greeting","goodbye","thanks","book_search"]

In [8]:

print('Welcome to the chatbot! Type "quit" to exit.')
while True:
    # get user input
    user_input = input('You: ').lower().strip()  #for user input run this line
    # user_input = "quit"

    # check if user wants to quit
    if user_input in mensajes_despedida:
        break

# encode user input text
    encoded_input = tokenizer.texts_to_sequences([user_input])
    padded_input = pad_sequences(encoded_input, maxlen=max_len, padding='post')

    # predict intent
    intent_prob = model.predict(padded_input)[0]
    intent_idx = np.argmax(intent_prob)
    intent_label = le.inverse_transform([intent_idx])[0]

    # retrieve response
    for intent in data['intents']:
        if intent['tag'] == intent_label:
          response = np.random.choice(intent['responses'])
          if intent_label in etiquetas_basicas:
            print('Chatbot:', response)
          else:
            print('Chatbot:', response['Book'],'. RATE: ', response['Rate'])

            print('Chatbot: Do you want a description of this book?')
            entrada = int(input('You:   1: YES,  if you dont want, press any key   '))

            if entrada==1:
              print('Chatbot:', response['Feedback'])

              print('\nChatbot: Do you have any question?')
            else:
              print('Chatbot: Do you have any question?')
            break

Welcome to the chatbot! Type "quit" to exit.
You: Hello
1/1 [==============================] - 0s 24ms/step
Chatbot: Hey! What brings you here today?
You: I'm looking for a book
1/1 [==============================] - 0s 21ms/step
Chatbot: Certainly! What genre are you interested in?
You: Recommend a history book
1/1 [==============================] - 0s 24ms/step
Chatbot: A Crack in the Edge of the World . RATE:  3.78
Chatbot: Do you want a description of this book?
You:   1: YES,  if you dont want, press any key   1
Chatbot: Unleashed by ancient geologic forces, a magnitude 8.25 earthquake rocked San Francisco in the early hours of April 18, 1906. Less than a minute later, the city lay in ruins. Bestselling author Simon Winchester brings his inimitable storytelling abilities to this extraordinary event, exploring the legendary earthquake and fires that spread horror across San Francisco and northern California in 1906 as well as its startling impact on American history and, just as im

In [ ]:
np.random.choice(intent['responses'])


In [ ]:
response['Book']

In [ ]:
response['Feedback']